In [1]:
import time
import zlib

import numpy as np
from numpy import random
import pandas as pd

from tqdm import tqdm
from faker import Faker

In [2]:
UNIQUE_VALUES = 40_000
ROW_COUNT = 5_000_000

In [3]:
faker_gen = Faker()

In [4]:
full_names = []
full_names_genenator = (faker_gen.name() for _ in range(UNIQUE_VALUES))

# Oh yes, we want to track progress
for name in tqdm(full_names_genenator):
    full_names.append(name)

40000it [00:03, 10920.85it/s]


In [5]:
index_values = random.choice(full_names, size=ROW_COUNT)

In [6]:
zipcode_generator = (faker_gen.zipcode() for _ in range(ROW_COUNT))
zipcodes = []

# Oh yes, we want to track progress
for zipcode in tqdm(zipcode_generator):
    zipcodes.append(zipcode)

5000000it [00:18, 264011.53it/s]


In [7]:
salary_changes = random.uniform(low=-5, high=20, size=ROW_COUNT)
salary_changes = np.around(salary_changes, decimals=1)

In [8]:
company_ids = random.randint(2**63, size=ROW_COUNT)

In [9]:
df = pd.DataFrame(
    index=index_values,
    data={"zipcode": zipcodes, "company_id": company_ids, "salary_change": salary_changes,
          "sort_id": list(range(ROW_COUNT))}
)

In [10]:
df.dtypes

zipcode           object
company_id         int64
salary_change    float64
sort_id            int64
dtype: object

In [11]:
df

,zipcode,company_id,salary_change,sort_id
Sarah Riggs,21996,2361990376624414571,-0.8,0
Matthew Jones,02512,6528427614962353275,10.3,1
Jason Smith,54238,8255113603324170872,3.8,2
Maurice Taylor,15685,298809915939219002,0.3,3
Gabriela Dominguez,32658,3425765616788486759,19.5,4
...,...,...,...,...
Robert Greene,80317,2642301740503478723,18.1,4999995
Ariel Yu,40585,5412565742354704424,-3.1,4999996
Mark Shepherd,14032,2298913774861761165,-1.8,4999997
Pamela Cowan,41828,5647069525706721049,11.9,4999998


In [12]:
RANDOM_ILOC = 4907
RANDOM_IDX = df.index[RANDOM_ILOC]
print(f"The random index is: `{RANDOM_IDX}`")

The random index is: `Karen Robinson`


In [13]:
%timeit df.loc[RANDOM_IDX]

68.8 ms ± 967 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%time sorted_df = df.reset_index().sort_values(["index", "sort_id"]).set_index("index")  # Stable sort

CPU times: user 6.48 s, sys: 620 ms, total: 7.1 s
Wall time: 7.1 s


In [15]:
%timeit sorted_df.loc[RANDOM_IDX]

183 µs ± 21.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
def custom_func(row) -> int:
    return zlib.crc32(row["zipcode"].encode("ascii") + str(row["company_id"]).encode("ascii"))

def ops_per_slice(mini_df: pd.DataFrame) -> pd.DataFrame:
    mini_df["hash"] = mini_df.apply(custom_func, axis=1)
    mini_df["prev_change"] = mini_df.salary_change.shift()
    return mini_df

In [17]:
def operations_on_df(input_df: pd.DataFrame, percent: int = 1) -> pd.DataFrame:
    assert percent > 0
    assert percent <= 100
    indexes = input_df.index.unique()

    # We just want a fraction of it for demo
    indexes = indexes[:int(percent * len(indexes) / 100)]
    
    result_dfs = []

    for index in tqdm(indexes):
        mini_df = input_df.loc[[index]]
        tmp_res = ops_per_slice(mini_df)
        result_dfs.append(tmp_res)
        
    return pd.concat(result_dfs)

In [18]:
%time operations_on_df(df)

100%|██████████| 331/331 [00:29<00:00, 11.17it/s]


CPU times: user 30 s, sys: 309 ms, total: 30.3 s
Wall time: 30.3 s


,zipcode,company_id,salary_change,sort_id,hash,prev_change
Sarah Riggs,21996,2361990376624414571,-0.8,0,189326387,NaN
Sarah Riggs,20981,3700974451310395564,6.0,1638,844534113,-0.8
Sarah Riggs,03052,8226229964558160138,18.5,104329,1310002665,6.0
Sarah Riggs,49524,5762822596102298036,17.3,164934,2620282093,18.5
Sarah Riggs,18438,5654063569205819625,15.0,259770,1665220583,17.3
...,...,...,...,...,...,...
Gina Gamble,25181,2095302418803573248,13.8,4860990,3727212992,3.4
Gina Gamble,45431,1894478488652672034,-0.2,4863649,2211409539,13.8
Gina Gamble,17772,3237888501089126040,1.1,4870636,1227281380,-0.2
Gina Gamble,08683,3132875107733488595,-2.3,4947059,2884680446,1.1


In [19]:
%time operations_on_df(sorted_df)

100%|██████████| 331/331 [00:02<00:00, 159.95it/s]


CPU times: user 3.38 s, sys: 85 ms, total: 3.47 s
Wall time: 3.44 s


,zipcode,company_id,salary_change,sort_id,hash,prev_change
index,,,,,,
Aaron Adkins,26185,9107979203059267955,12.5,2051,3850086661,NaN
Aaron Adkins,80654,9212690069555363938,-5.0,16126,1448707006,12.5
Aaron Adkins,24452,6934899145183938796,11.4,22357,1899769728,-5.0
Aaron Adkins,33239,3076589748400004361,-3.7,36872,3857436049,11.4
Aaron Adkins,99785,6286608026656905232,7.9,46287,987917749,-3.7
...,...,...,...,...,...,...
Adrienne Schmidt,08881,8657856856470075209,18.3,4940097,3642743413,18.2
Adrienne Schmidt,25511,5381684259270911311,-3.0,4954568,577931621,18.3
Adrienne Schmidt,06589,3271103089986577376,-2.9,4974531,3248851887,-3.0
